In [8]:
%matplotlib inline

In [23]:
import numpy as np
from numpy import linalg as LA
import os, pickle, gzip

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as plticker
from matplotlib import cm
import gzip

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

KITTI_HOME = '/home/kreimer/KITTI'

In [44]:
def read_dts(file):
    dts = []
    with open(file, 'r') as f:
        for num, line in enumerate(f):
            pose = np.array([float(val) for val in line.split()]).reshape(3,4)
            pose = np.vstack((pose, [0, 0, 0, 1]))
            try:
                delta = np.dot(LA.inv(ppose), pose)
            except NameError:
                ppose = pose
                continue
            dt = delta[:3,3]
            dts.append(dt)

            ppose = pose
    return dts

In [59]:
seq = 8
poses_file = os.path.join('data', 'paths', '%02d.txt' % seq)

dts = np.array(read_dts(poses_file))
scale = LA.norm(dts[0,:])

print('read a total of', dts.shape[0], 'scales from', poses_file)
print('ground truth scale:', scale, )

y = LA.norm(dts, axis=1) / scale

# load the precomputed features
with gzip.open(os.path.join('data', 'features', 'un_%02d_1_6_4_100.pklz' % seq), 'rb') as fd:
    pairs, feat, dts = pickle.load(fd)

# load the model
with open(os.path.join('data', 'models', 'rf_un_1_2_6_4_100.pkl'), 'rb') as fd:
    rf = pickle.load(fd)

X = np.array(feat)

# predict
y_pred = rf.predict(X)

# compute scale estimate
scale_est = np.dot(y_pred, y).sum()/np.dot(y, y).sum()
print('estimated scale', scale_est)

read a total of 4069 scales from data/paths/08.txt
ground truth scale: 0.776692622852


AttributeError: 'RandomForestRegressor' object has no attribute '_base_estimator_'

In [69]:
t1 = rf.estimators_[0]
print(dir(t1))
print(rf.apply(X[1,:]))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_validate_X_predict', 'apply', 'class_weight', 'classes_', 'criterion', 'decision_path', 'feature_importances_', 'fit', 'fit_transform', 'get_params', 'max_depth', 'max_features', 'max_features_', 'max_leaf_nodes', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_classes_', 'n_features_', 'n_outputs_', 'predict', 'presort', 'random_state', 'score', 'set_params', 'splitter', 'transform', 'tree_']
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   2 tasks      | e

/home/kreimer/prj/monos/.env/lib/python3.5/site-packages/sklearn/utils/validation.py:394: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
